In [2]:
!pip install --quiet pandas
!pip install --quiet numpy

import pandas as pd
import numpy as np

####### Questions & Assumptions #######
# max Rating by counterparty? >> Global for each counterparty only or by ['legal_entity', 'counterparty', 'tier']
# How to handle NaN when grouping and summing values by status? Assumed assigning it 0
# How to group the data? Assumed by ['legal_entity', 'counterparty', 'tier']
# New Total Recods? assumed saved in a separate file. Could easly be merged with the final dataset if required
# Output file type? Assumed CSV
# Although it is not best practice (harder to manipulate columns), 
#I assumed column names in the requested output file is mandatory, thus renamed everything to match it

# Reading the two csv files
ds1 = pd.read_csv('../dataset1.csv')
ds2 = pd.read_csv('../dataset2.csv')

# Joining the datasets on 'counter_party' key and renaming 'counter_party' column to match the required output
df_joined = ds1.merge(ds2, how='left', on='counter_party').rename(columns={'counter_party': 'counterparty'})

# This step divides the values into two seperate columns according to status type 
# and assigning 0 when no value is found
df1 = df_joined.assign(
    sum_value_where_status_ARAP = np.where(df_joined['status']=='ARAP', df_joined['value'], 0),
    sum_value_where_status_ACCR = np.where(df_joined['status']=='ACCR', df_joined['value'], 0)  
)

# Since I am not sure if the max Rating by counterparty requested in the assignement 
# wants the global max of rating in each counterparty only
# or in the combination of ['legal_entity', 'counterparty', 'tier'], I will do both separately, starting with
# getting max rating in ['legal_entity', 'counterparty', 'tier'] in route 1, and then changing their values with
# global max of rating in each counterparty only in route 2

##### Route 1: max of rating in ['legal_entity', 'counterparty', 'tier'] groups
df = df1.groupby(['legal_entity', 'counterparty', 'tier']).agg({
    'rating':max,
    'sum_value_where_status_ARAP':sum, 
    'sum_value_where_status_ACCR':sum
}).reset_index()

# Renaming columns to match final output file format (not best practice)
df.rename(columns={
    'rating':'max(rating by counterparty)',
    'sum_value_where_status_ARAP':'sum(value where status=ARAP)', 
    'sum_value_where_status_ACCR':'sum(value where status=ACCR)'
}, inplace=True)

# Saving Route 1 to CSV, uncomment below line
#df.to_csv('outputFile_maxRatingByLegalEntityCounterpartyTier', index=False)
print(f"####### outputFile_maxRatingByLegalEntityCounterpartyTier ####### \n\n{df}\n\n")



##### Route 2: global max of rating in each counterparty only
df_maxRatingByCounterpartyOnly = df_joined.groupby('counterparty')['rating'].max()

# Merge the new max with the dataframe from route 1 and put the new values in 'max(rating by counterparty)' col
df['max(rating by counterparty)'] = df.merge(df_maxRatingByCounterpartyOnly, how='left', on='counterparty')['rating']

# Saving Route 2 to CSV, uncomment below line
#df.to_csv('outputFile_maxRatingByCounterpartyOnly', index=False)
print(f"####### outputFile_maxRatingByCounterpartyOnly ####### \n\n{df}\n\n")

##### Create new record to add total for each of legal entity, counterparty & tier
# Assuming in its own output file as count is not part of the mentioned col in final requested output file
df_grouped_count = df_joined.groupby(['legal_entity', 'counterparty', 'tier']).size().reset_index(name='count')

# Saving to CSV, uncomment below line
#df_grouped_count.to_csv('total_record', index=False)
df_grouped_count

####### outputFile_maxRatingByLegalEntityCounterpartyTier ####### 

  legal_entity counterparty  tier  max(rating by counterparty)  \
0           L1           C1     1                            3   
1           L1           C3     3                            6   
2           L1           C4     4                            6   
3           L2           C2     2                            3   
4           L2           C3     3                            2   
5           L2           C5     5                            6   
6           L3           C3     3                            4   
7           L3           C6     6                            6   

   sum(value where status=ARAP)  sum(value where status=ACCR)  
0                            40                             0  
1                             5                             0  
2                            40                           100  
3                            20                            40  
4                

,legal_entity,counterparty,tier,count
0,L1,C1,1,3
1,L1,C3,3,1
2,L1,C4,4,2
3,L2,C2,2,2
4,L2,C3,3,1
5,L2,C5,5,3
6,L3,C3,3,3
7,L3,C6,6,3
